# Modular RAG  - Raptor RAG (2024, Stanford Univ.)

- Tackling Problem: RAG에 들어가는 문서 수가 많아질수록, 그리고 답안에 필요한 정보량이 long-tail로 방대해야 할때 Retrieval 성능이 너무 좋지 않아진다

- 제안 방법론: 정보들을 단계적 추상화를 통해 enrich시켜서 질문에서 요구하는 context의 뎁스에 알맞는 청크가 뽑히도록 하자.

- (인덱싱 프로세스) Text Chunk를 의미적으로 유사한 다른 Chunk들과 군집화 시키고 -> 군집별 요약문 생성하여 정보 압축 -> 요약문에 대한 임베딩 생성 (전체 프로세스 n번 반복하여 계층적 트리구조 생성)

- (검색 프로세스) Query와 가장 유사한 정보를 Root Node(Layer)에서 찾고, 그 root node가 생성한 요약문이 기반으로 두는 child summary를 찾고, 반복하며 최종적으로 사용자가 지정한 n개의 top-k 유사한 노드가 뽑힐때까지 트리 구조를 traverse함.

# RAPTOR RAG 구조

### Indexing 구조

- 특이점은, GMM기반의 군집화를 하기 때문에 클러스터별 해당하는 노드의 개수가 다르고, 각 차일드노드가 속할 수 있는 패런트노드도 1대다수 관계일 수 있는데, 정보체계라는걸 생각해보면 이 클러스터링 알고리즘이 굉장히 적합한 것으로 사료됨.

# Modular RAG(RAPTOR) 간단구성

- 관련 패키지 임포트
- 활용 LLM api정보 설정
- RAG 파이프라인 글로벌 세팅 설정
- 기준 데이터셋 로드
- 기준 데이터셋 전처리
- (베이스RAG)벡터스토어인덱스 설정
- (베이스RAG)쿼리 엔진 설정
- (RAPTOR RAG) 크로마DB 설정
- (RAPTOR RAG) 쿼리 엔진 설정

In [ ]:
!pip install llama_index OpenAI chromadb datasets llama-index-vector-stores-chroma llama-index-packs-raptor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
# API 설정
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 관련 패키지 임포트
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex
)
import os
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.packs.raptor import RaptorPack
from llama_index.core.query_engine import RetrieverQueryEngine
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# RAG 파이프라인 글로벌 설정
Settings.llm=OpenAI(model='gpt-4o-mini',temperature=0)
Settings.embed_model=OpenAIEmbedding(model='text-embedding-3-small')

In [ ]:
# 데이터 로드
dataset = SimpleDirectoryReader(input_files=['/content/cinderella.txt']).load_data()

In [ ]:
# 청크사이즈 전처리 파이프라인 생성
pipeline_200 = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ]
)

# 설정한 파이프라인 설정으로 기준 다큐먼트 노드 오브젝트로 변경
nodes_200 = pipeline_200.run(documents=dataset)

In [ ]:
#기준 데이터 확인
nodes_200

[TextNode(id_='ecda91d6-f6ac-4e9e-a692-42246ecc7324', embedding=None, metadata={'file_path': '/content/cinderella.txt', 'file_name': 'cinderella.txt', 'file_type': 'text/plain', 'file_size': 20547, 'creation_date': '2024-08-09', 'last_modified_date': '2024-08-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='92878b71-511b-4acb-8f77-b21cc6009263', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/cinderella.txt', 'file_name': 'cinderella.txt', 'file_type': 'text/plain', 'file_size': 20547, 'creation_date': '2024-08-09', 'last_modified_date': '2024-08-09'}, hash='e2eb5b8d689903a4d203f2da7145df09065e3f55e6f1c795742f91740e67f7dc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='

# Naive-RAG 로 베이스 모델 구축

In [ ]:
# 벡터스토어 인덱스 설정
vector_index_200 = VectorStoreIndex(nodes_200)

# 쿼리 엔진 설정
query_engine_200 = vector_index_200.as_query_engine(similarity_top_k=2)

In [ ]:
from pprint import pprint

In [ ]:

response = query_engine_200.query("How does Cinderella find a happy ending?")


In [ ]:
pprint(response.response)

('Cinderella finds a happy ending through the intervention of her faëry '
 'godmother, who transforms her appearance and provides her with a means to '
 'attend the ball. Despite her sisters not recognizing her, she captures the '
 'attention of the prince. After leaving the ball at midnight, she '
 'accidentally drops a glass slipper, which the prince later uses to search '
 "for her. Ultimately, the prince's determination to find the owner of the "
 'slipper leads to a reunion with Cinderella, allowing her to escape her '
 'previous hardships and find happiness.')


In [ ]:
# hallucination이 없을까?
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('Cinderella drew near her sisters, and frequently spoke to them; and in\n'
 'her goodness of heart she offered them the delicacies which she had\n'
 'received from the prince:  but they did not know she was their sister.\n'
 '\n'
 'When Cinderella heard the clock strike three-quarters past eleven, she\n'
 'made a low courtesy to the whole assembly and retired in haste.\n'
 '\n'
 'You see how fortune befriends the good-hearted, and even out of such\n'
 'unpromising material as a pumpkin and mice, can make a coach and six,\n'
 'with which to honor her worthy favorite. So Cinderella goes to the ball;\n'
 'but to teach her to be diligent and faithful in her engagements, her\n'
 'faëry godmother enjoins upon her that she return home at twelve.')
('Never mind--her heart is in the right place--she\n'
 'is a charming good creature; and although virtue goes home in rags, it\n'
 'will leave some token behind--some foot-print by which it can be known\n'
 'and traced wherever it has once walked. 

In [ ]:
# 일단 retrieved context로만 답하게 만들어보자
response = query_engine_200.query(""" You have a long term memory issue. You are not able to answer anything unless the context is provided. When user asks the question, you will be provided with 2 context chunks that are relevant to answer the questions.
                                  answer the question based on the context provided strictly.

                                  Question:
                                  How does Cinderella find a happy ending?
                                  Answer:
                                  """)

In [ ]:
pprint(response.response)

('Cinderella finds a happy ending through the recognition of her virtue and '
 'the magical events that unfold after she loses her glass slipper at the '
 'ball. Despite returning home in rags, her goodness leaves a lasting '
 'impression, and the prince, enamored by her beauty and charm, searches for '
 'the owner of the slipper. This ultimately leads to her being reunited with '
 'the prince, allowing her to escape her previous hardships and find '
 'happiness.')


In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('The sentinels at the palace gate were closely questioned as to whether\n'
 'they had not seen a princess coming out; but they answered they had seen\n'
 'no one except a shabbily dressed girl, who appeared to be a peasant\n'
 'rather than a young lady.\n'
 '\n'
 'On this second night, as you have taken notice, dazzled by worldly show\n'
 'and the pleasing flattery of her royal lover, Cinderella over-stays her\n'
 "time, and is compelled to make her way back to her father's house on\n"
 'foot and in rags--an everlasting lesson to all the pretty little\n'
 'Cinderellas in the world to keep their word, and to act in good faith by\n'
 'such as befriend them.')
('Never mind--her heart is in the right place--she\n'
 'is a charming good creature; and although virtue goes home in rags, it\n'
 'will leave some token behind--some foot-print by which it can be known\n'
 'and traced wherever it has once walked. We shall hear from that little\n'
 'lost glass slipper again!\n'
 '\n'
 'When the two

# RAPTOR RAG로 QA 성능 고도화

In [ ]:
# RAPTOR RAG 구현용 크로마DB 활용
client = chromadb.PersistentClient(path="./cinderella_db")
collection = client.get_or_create_collection("cinderella")

vector_store = ChromaVectorStore(chroma_collection=collection)

In [ ]:
# 라마인덱스 랩터팩(통합팩) 사용하여 아키 세팅
raptor_pack = RaptorPack(
    dataset,
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ),
    llm=OpenAI(model="gpt-4o-mini", temperature=0),
    vector_store=vector_store,
    similarity_top_k=2,
    mode="collapsed",
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ],
)

Generating embeddings for level 0.
Performing clustering for level 0.
Generating summaries for level 0 with 5 clusters.
Level 0 created summaries/clusters: 5
Generating embeddings for level 1.
Performing clustering for level 1.
Generating summaries for level 1 with 1 clusters.
Level 1 created summaries/clusters: 1
Generating embeddings for level 2.
Performing clustering for level 2.
Generating summaries for level 2 with 1 clusters.
Level 2 created summaries/clusters: 1


In [ ]:
# 생성된 인덱스 기반 Raptor Retriever 엔진 정의
raptor_retriever = raptor_pack.retriever

raptor_query_engine = RetrieverQueryEngine.from_args(
    raptor_retriever, llm=OpenAI(model="gpt-4o-mini", temperature=0)
)

In [ ]:
# 시험
response = raptor_query_engine.query("How does Cinderella find a happy ending?")

In [ ]:
pprint(response.response)

('Cinderella finds a happy ending by forgiving her step-sisters for their '
 'mistreatment and marrying the prince, who recognizes her as his true love '
 'after she fits the glass slipper. Additionally, she shows her kindness by '
 'allowing her step-sisters to live in the palace and arranging marriages for '
 'them with two lords.')


In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('Cinderella, a kind-hearted young woman mistreated by her cruel step-sisters, '
 'longs to attend a royal ball. With the help of her fairy godmother, she is '
 'magically transformed and arrives at the palace in a splendid carriage, '
 'captivating the prince and the guests with her beauty and grace. Despite her '
 "step-sisters' mockery, Cinderella dances with the prince, who is enchanted "
 'by her.\n'
 '\n'
 'After a magical evening, she flees the ball at midnight, leaving behind a '
 'glass slipper. The prince searches for the owner of the slipper, trying it '
 'on various noblewomen, including her step-sisters, but to no avail. When '
 'Cinderella tries the slipper, it fits perfectly, and she reveals a matching '
 'slipper. Her fairy godmother enhances her appearance, confirming her as the '
 "prince's true love.\n"
 '\n'
 'Cinderella forgives her step-sisters for their past mistreatment and marries '
 'the prince, allowing them to live in the palace and marrying them off to two 

In [ ]:
import pandas as pd
data = pd.read_csv('/content/korean_webtext.csv').iloc[:,1:]

# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
documents = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    documents.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
# RAPTOR RAG 구현용 크로마DB 활용
client = chromadb.PersistentClient(path="./wiki_db")
collection = client.get_or_create_collection("wiki")

vector_store = ChromaVectorStore(chroma_collection=collection)

In [ ]:
# 라마인덱스 랩터팩(통합팩) 사용하여 아키 세팅
raptor_pack = RaptorPack(
    documents,
    embed_model=OpenAIEmbedding(
        model="text-embedding-3-small"
    ),
    llm=OpenAI(model="gpt-4o-mini", temperature=0),
    vector_store=vector_store,
    similarity_top_k=2,
    mode="collapsed",
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ],
)

Generating embeddings for level 0.
Performing clustering for level 0.
Generating summaries for level 0 with 85 clusters.
Level 0 created summaries/clusters: 85
Generating embeddings for level 1.
Performing clustering for level 1.
Generating summaries for level 1 with 18 clusters.
Level 1 created summaries/clusters: 18
Generating embeddings for level 2.
Performing clustering for level 2.
Generating summaries for level 2 with 5 clusters.
Level 2 created summaries/clusters: 5


In [ ]:
# 생성된 인덱스 기반 Raptor Retriever 엔진 정의
raptor_retriever = raptor_pack.retriever

raptor_query_engine = RetrieverQueryEngine.from_args(
    raptor_retriever, llm=OpenAI(model="gpt-4o-mini", temperature=0)
)

In [ ]:
# 청크사이즈 전처리 파이프라인 생성
pipeline_200 = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ]
)

# 설정한 파이프라인 설정으로 기준 다큐먼트 노드 오브젝트로 변경
nodes_200 = pipeline_200.run(documents=documents)

In [ ]:
# 벡터스토어 인덱스 설정
vector_index_200 = VectorStoreIndex(nodes_200)

# 쿼리 엔진 설정
query_engine_200 = vector_index_200.as_query_engine(similarity_top_k=2)

In [ ]:
ds.to_pandas().iloc[16].text

'익숙하지 않은 것이 주는 낯설고 설레는 경험, 그것을 통해 얻는 영감은 여행을 하는 이유가 된다. 특별한 것으로 잘 꾸며진 숙소는 일상에서 쉽게 접할 수 없기에 언제나 여행의 로망으로 손꼽힌다. 상위 1%의 프리미엄을 표방하는 리조트인 제주 롯데 아트빌라스는 이런 로망의 궁극을 채워주는 곳이다. 세계적인 건축 거장 5인이 제주 자연을 모티브 삼아 완성한 건축물이 압도적이다. 한라산과 바다를 잇는 제주 지형 속 수직과 수평을 조화롭게 담아낸 승효상, 해안선 등을 모티프로 생명력이 느껴지는 공간을 선보인 도미니크 페로, 바람이 자연스럽게 오갈 수 있는 구조를 통해 천혜의 제주를 가득 안고자 한 이종호와 제주 오름을 형상화해 현무암으로 덮은 지붕 등 과감한 시도를 한 켄고 쿠마가 그 주인공. 여기에 국내 건축 집단인 DA그룹에서 선보인 리듬감 넘치는 스카이라인의 건축까지 더해져 구역별로 서로 다른 매력을 담아낸 건축물들이 위용을 뽐낸다. 리조트는 크게 건축가 별로 나뉜 A~E 블록과 509㎡(약 154평) 규모의 펜트하우스 한 채로 구성됐다. 대부분 넓은 규모라 대가족 혹은 여러 명 단위의 여행에 잘 맞다. 아늑하고 평온한 분위기 속에 검은 현무암, 하늘과 바다가 만나는 풍경, 정갈한 초록의 조경 등 특유의 제주스러운 풍경을 한껏 담은 이곳. 특별한 건축물과 휴식에 최적화된 분위기가 더없이 매력적이다. 보안 검색을 지나 리조트 내부로 들어오면 커뮤니티 센터가 가장 먼저 맞이해준다. 이곳은 전체 리조트의 센터 역할을 하는 곳이다. 체크인을 하는 곳임은 물론 조식을 위한 레스토랑 및 라운지, 야외 수영장과 사우나, 피트니스센터 등 다양한 부대시설이 두루 자리하고 있다. 커뮤니티 센터 위로는 언덕 구조를 활용한 전망대도 있다. 건물 뒤편으로 잔디가 깔린 언덕을 따라 올라가면 된다. 중문 일대와 저 멀리 서귀포 바다까지 넓게 펼쳐진 풍경을 보노라면 천혜의 제주 품 속에 머물고 있다는 것을 새삼 실감할 수 있다. 보는 것만으로도 압도하는 휴식의 공간, 아트빌라스 속 펜트하

In [ ]:
naive_response = query_engine_200.query("분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?")

In [ ]:
pprint(naive_response.response)

('해외에서 분당과 같은 신도시가 성공하기 어려운 이유는 현지의 문화, 시장, 사람들, 관계, 제도 등을 충분히 이해하지 못하기 때문입니다. '
 '국내식 접근 방식으로는 로컬 환경에 적합한 해결책을 찾기 힘들며, 각 지역의 특성을 반영한 맞춤형 전략이 필요합니다. 성공적인 신도시 '
 '개발은 이러한 로컬 이해를 바탕으로 이루어져야 합니다.')


In [ ]:
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

'그런데 어디에도 제대로된 한국적 신도시 성공담이 들리지 않는다. 오히려 고전을 면치못하는 게 현실이다.'
('우리나라가 아무리 국내에서 많은 신도시를 지었다고 해도 해외신도시개발은 다른 얘기다. 결코 만만하게 봐서는 안될것이다. 로마에 가면 '
 '로마법을 따라야하듯 해외에서 국내식 접근은 잊기 바란다. 로컬을 잘 이해하는 것이 성공의 첩경이다. 진출할 도시의 사람, 문화, 시장, '
 '관계, 제도 등을 잘 이해하는 것이 큰 사업의 가장 기초이자 성공의 결정요인이다.')


In [ ]:
# 시험
response = raptor_query_engine.query("분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?")

In [ ]:
pprint(response.response)

('해외에서 분당과 같은 신도시가 성공하기 어려운 이유는 현지의 문화, 시장, 사람들, 관계, 제도 등을 충분히 이해하지 못하기 때문입니다. '
 '국내식 접근 방식으로는 로컬 환경에 적합한 해결책을 찾기 힘들고, 각 지역의 특성을 반영하지 못하면 성공하기 어렵습니다. 따라서, '
 '성공적인 신도시 개발을 위해서는 해당 지역의 로컬 요소를 잘 이해하는 것이 필수적입니다.')


In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

'그런데 어디에도 제대로된 한국적 신도시 성공담이 들리지 않는다. 오히려 고전을 면치못하는 게 현실이다.'
('우리나라가 아무리 국내에서 많은 신도시를 지었다고 해도 해외신도시개발은 다른 얘기다. 결코 만만하게 봐서는 안될것이다. 로마에 가면 '
 '로마법을 따라야하듯 해외에서 국내식 접근은 잊기 바란다. 로컬을 잘 이해하는 것이 성공의 첩경이다. 진출할 도시의 사람, 문화, 시장, '
 '관계, 제도 등을 잘 이해하는 것이 큰 사업의 가장 기초이자 성공의 결정요인이다.')


In [ ]:
naive_response = query_engine_200.query("캐나다가 주목받는 국가인 이유에 대해 알려줘")

In [ ]:
pprint(naive_response.response)

('캐나다는 살기 좋은 나라와 유학하기 가장 좋은 나라로 높은 평가를 받고 있으며, 일, 방문, 생활 측면에서도 최고의 국가로 자리매김하고 '
 '있습니다. 이러한 이유로 많은 사람들이 캐나다를 주목하고 있습니다. 또한, 캐나다의 우수한 교육 프로그램, 아름다운 자연, 다문화 사회, '
 '저렴한 생활비 등이 유학생들에게 매력적으로 작용하고 있습니다.')


In [ ]:
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

('캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀을 집으로 가져 왔을뿐만 아니라 캐나다는 "살기 좋은 '
 '나라"에서 "유학하기 가장 좋은 나라"로 분류하는 데있어 최고의 자리를 보장하는 동시에 최고의 국가 일, 방문 및 생활로 자리 '
 '매김했습니다. 당신의 배낭에 단풍잎을 꿰매는 진정한 동기를 찾고 있습니까?')
('그러나 최근 몇 년 동안 점점 더 인기를 얻고있는 한 가지 선택이 있습니다. 캐나다 대학에서 교육을 추구하는 것입니다. 많은 유학생들이 '
 '캐나다를 유학 목적지로 선택하고 있으며 그 이유는 당연합니다. 캐나다는 우수한 교육 프로그램, 경이로운 자연, 다문화 사회 및 저렴한 '
 '생활비를 제공합니다.')


In [ ]:
# 시험
response = raptor_query_engine.query("캐나다가 주목받는 국가인 이유에 대해 알려줘")

In [ ]:
pprint(response.response)

('캐나다는 우수한 교육 프로그램과 아름다운 자연, 다문화 사회, 저렴한 생활비 덕분에 주목받고 있습니다. 또한, "살기 좋은 나라"와 '
 '"유학하기 가장 좋은 나라"로 높은 평가를 받으며, 많은 유학생들이 이곳을 선택하고 있습니다. 캐나다에는 약 96개의 대학이 있으며, '
 '국제적인 교육의 혜택이 강조되고 있습니다.')


In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀을 집으로 가져 왔을뿐만 아니라 캐나다는 "살기 좋은 '
 '나라"에서 "유학하기 가장 좋은 나라"로 분류하는 데있어 최고의 자리를 보장하는 동시에 최고의 국가 일, 방문 및 생활로 자리 '
 '매김했습니다. 당신의 배낭에 단풍잎을 꿰매는 진정한 동기를 찾고 있습니까?')
('캐나다는 3년 연속 세계에서 가장 주목할 만한 국가로 선정되었으며, "살기 좋은 나라"와 "유학하기 가장 좋은 나라"로도 높은 평가를 '
 '받고 있습니다. 많은 유학생들이 캐나다를 선택하는 이유는 우수한 교육 프로그램, 아름다운 자연, 다문화 사회, 저렴한 생활비 때문입니다. '
 '캐나다에는 약 96개의 대학이 있으며, 국제적인 교육의 혜택이 강조됩니다. 2018년에는 세계에서 가장 국제적인 200개 대학 목록에 '
 '7개의 캐나다 대학이 포함되었습니다. 캐나다의 공식 언어는 영어와 프랑스어이며, 각각 56.9%와 21.3%의 인구가 모국어로 '
 '사용합니다.')


In [ ]:
naive_response = query_engine_200.query("한국과 일본의 유명한 해군 제독에 대해 알려줘")

In [ ]:
pprint(naive_response.response)

('한국과 일본의 유명한 해군 제독에 대한 정보는 제공되지 않았습니다. 그러나 일본의 경우, 마카로프 제독이 언급되며, 그는 러시아 해군의 '
 '전략가로서 해전에서 중요한 역할을 했습니다. 한국의 해군 제독에 대한 구체적인 정보는 포함되어 있지 않습니다.')


In [ ]:
# 참조 컨텍스트 확인
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

('일본으로부터 막대한 영향을 받고 영향을 주기도 하며 서로가 공존하는 모습을 두고 한국은 선, 일본은 악이라는 이분법적 조센징의 머가리 '
 '구조가 우스꽝 스러워 보일뿐이다.')
('당시 러시아는 세계에서 영국에 이은 해군 강국으로 해군의 어뢰정 운용과 전술에 있어 최고로 인정 받고 있었으며, 해군 전술 전략에 있어 '
 '세계적으로 존경을 받고 있는 마카로프 제독과 같은 전략가가 있었다. 마카로프 제독은 1904년 일러전쟁 발발 직 후 여순항 해전 중 일본 '
 '연함 함대를 발견하고 쫓아가다 기뢰에 걸려 파편에 맞아 즉사했다.')


In [ ]:
# 시험
response = raptor_query_engine.query("한국과 일본의 유명한 해군 제독에 대해 알려줘")

In [ ]:
pprint(response.response)

('한국의 유명한 해군 제독은 이순신입니다. 그는 임진왜란 동안 일본 해군에 대한 여러 차례의 결정적인 승리를 이끌어냈으며, 그의 독창적인 '
 "전술인 '학익진'은 특히 주목받았습니다. 이순신의 전투에서의 성공은 일본 해군이 일시적으로 해상 전쟁을 포기하게 만드는 결과를 "
 '가져왔습니다. \n'
 '\n'
 '일본의 유명한 해군 제독은 도고 헤이하치로입니다. 그는 러일 전쟁 중 쓰시마 해전에서 러시아에 대한 중요한 승리를 거두었으며, 이순신의 '
 '전술에 깊은 존경심을 가지고 그의 전략을 연구하고 일본 해군의 현대화에 기여했습니다. 도고는 이순신을 idolize하며 그의 전술을 일본 '
 '해군 교육에 포함시켰고, 이순신을 기리기 위해 신사를 세우고 바다에서의 승리를 기원하는 의식을 행하기도 했습니다.')


In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('The text covers various historical and contemporary aspects of the '
 'relationship between Korea and Japan, focusing on military, sports, and '
 'economic comparisons. \n'
 '\n'
 'It begins with the Tsushima Naval Battle during the Russo-Japanese War, '
 'where Japan, under Admiral Togo Heihachiro, achieved a significant victory '
 "over Russia, showcasing Japan's military prowess. This battle resulted in "
 "heavy losses for Russia, while Japan suffered minimal casualties. Togo's "
 'tactics, inspired by Admiral Yi Sun-sin of Korea, became foundational in '
 'modern naval warfare.\n'
 '\n'
 "The narrative then shifts to the Imjin War, highlighting Yi Sun-sin's "
 "decisive victories against Japan, which led to Japan's temporary abandonment "
 'of naval warfare. Yi\'s innovative strategies, particularly the "Hakkikjin" '
 'formation, earned him respect in Japan, where his tactics were studied and '
 'revered, influencing Japanese naval practices.\n'
 '\n'
 'In the realm of sports, t

In [ ]:
data.to_csv('korean_webtext.csv')

,text,source,token_count,__index_level_0__
0,사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 ...,oscar2201,3348,0
1,“아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노...,oscar2201,1427,1
2,일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다...,oscar2201,2458,2
3,"재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 ...",oscar2201,2838,3
4,"상급자의 범죄와 비리, 부패를 하급자에게 돌리는 것으로 따지면 타의추종을 불허하는 ...",oscar2201,1628,4
5,최근 언론보도에 의하면 이재현 CJ그룹 회장이 지난해 말 두 자녀에게 증여하였던 주...,oscar2201,1366,5
6,"나는 노무현의 시대를 살지 않았다. 그러니까, 나는 이 땅의 생명체로 살아있긴 했지...",oscar2201,2017,6
7,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,oscar2201,1421,7
8,"안녕하세요. 한화솔루션입니다. 지난주, 슬기로운 솔루션 직장생활 2탄에 이어 이번엔...",oscar2201,2143,8
9,캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀...,oscar2201,1104,9
